In [ ]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q datasets
# !pip install -U adapter-transformers sentencepiece
!pip install -U adapter-transformers sentencepiece
!pip install datasets
!pip install evaluate==0.4.0
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer

def model_and_tokenizer(model_name):
  if model_name =="flan-t5":
    base_model = "google/flan-t5-base"
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    return base_model, tokenizer


# tokenize the dataset
def encode_batch(examples, text_column = 'document', summary_column ='summary', padding = "max_length"):
    # convert to lists of strings
    inputs, targets = [], []
    for i in range(len(examples[text_column])):
        if examples[text_column][i] and examples[summary_column][i]:
            inputs.append(examples[text_column][i])
            targets.append(examples[summary_column][i])

    # add prefix to inputs
    inputs = [prefix + inp for inp in inputs]

    # finally we can tokenize the inputs and targets
    model_inputs = tokenizer(inputs, max_length=512, padding=padding, truncation=True)
    labels = tokenizer(targets, max_length=512, padding=padding, truncation=True)

    # rename to labels for training
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

def load_split(dataset, split_name, max_items):
    # load the split
    #dataset = load_dataset("xsum")[split_name]
    dataset = dataset[split_name]
    # only use the first max_items items
    dataset = dataset.filter(lambda _, idx: idx < max_items, with_indices=True)
    # tokenize the dataset
    dataset = dataset.map(
        encode_batch,#(examples, model = model_name ,text_column = 'document', summary_column ='summary'),
        batched=True,
        remove_columns=dataset.column_names,
        desc="Running tokenizer on " + split_name + " dataset",
    )
    # set the format to torch
    dataset.set_format(type="torch", columns=["input_ids", "labels"])

    return dataset

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from transformers.adapters import LoRAConfig
import numpy as np
from datasets import load_dataset

model_name = 'flan-t5'
prefix = 'summarize: '

dataset = load_dataset('csv', data_files={'train': 'df_train.csv',
                                              'test': 'df_test.csv'})

base_model, tokenizer = model_and_tokenizer(model_name)

# start with the pretrained base model
model = AutoModelForSeq2SeqLM.from_pretrained(
    base_model
)

# set the parameters for LoRA
config = LoRAConfig(
    r=8,
    alpha=16,
    # use it on all of the layers
    intermediate_lora=True,
    output_lora=True
)

# make a new adapter for the XSum dataset
model.add_adapter("summarization", config=config)
# enable the adapter for training
model.train_adapter("summarization")
model.set_active_adapters(["summarization"])

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import TrainingArguments, AdapterTrainer, TrainerCallback
from datasets import load_dataset

# small batch size to fit in memory
batch_size = 1

training_args = TrainingArguments(
    learning_rate=3e-4,
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    remove_unused_columns=False
)

# create the trainer
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    # load the dataset
    train_dataset=load_split(dataset, "train", 10),
    eval_dataset=load_split(dataset, "test", 2),
)

Running tokenizer on train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Running tokenizer on test dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 30
  Number of trainable parameters = 1966080
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=30, training_loss=45.22047526041667, metrics={'train_runtime': 11.7227, 'train_samples_per_second': 2.559, 'train_steps_per_second': 2.559, 'total_flos': 20723914506240.0, 'train_loss': 45.22047526041667, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2
  Batch size = 1


{'eval_loss': 43.63471221923828,
 'eval_runtime': 0.3206,
 'eval_samples_per_second': 6.238,
 'eval_steps_per_second': 6.238,
 'epoch': 3.0}

In [ ]:
num_validation = 10
val = load_dataset('csv', data_files={'validation': 'df_validation.csv'})

validation_dataset = load_split(val, 'validation', num_validation)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/10 [00:00<?, ? examples/s]

Running tokenizer on validation dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
predictions, references = [] , []

for i in range(num_validation):
    # load the input and label
    input_ids = validation_dataset[i]['input_ids'].unsqueeze(0).to(0)
    label_ids = validation_dataset[i]['labels'].unsqueeze(0).to(0)
    # use the model to generate the output
    output = model.generate(input_ids, max_length=1024)
    # convert the tokens to text
    input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    label_text = tokenizer.decode(label_ids[0], skip_special_tokens=True)
    predictions.append(output_text)
    references.append(label_text)

    print('Input:', input_text)
    print('Output:', output_text)
    print('Label:', label_text)
    print('---')

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



Input: summarize: The University and College Union says the 1.1% rise offered by the universities is "an insult". But the Universities and Colleges Employers Association said the walkout was "disappointing given the very good pay offer". Unions representing university support staff are balloting on the offer, with strike action possible in the autumn. UCU says its members have suffered a real-terms pay cut of 14.% since 2009 and complains the squeeze on staff salaries has come as university leaders enjoyed hefty increases. "A 1.1% pay offer is an insult to hardworking staff, especially in light of the 5% pay rise vice-chancellors have enjoyed while holding down staff pay," said general secretary Sally Hunt. "Industrial action which impacts on students is never taken lightly, but members feel that they have been left with no alternative. "If the employers wish to see a swift end to this dispute, and avoid further disruption, they need to come back to the table with a much-improved offer

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



Input: summarize: The government has "called in" proposals to shut St Joseph's Primary in Milngavie near Glasgow. The council plans to build a new denominational primary in nearby Bearsden - replacing both St Joseph's and the school on the site at present. Parents at St Joseph's and the Catholic Church have been fighting the plan. East Dunbartonshire Council plans to merge St Joseph's Primary with St Andrew's Primary in neighbouring Bearsden. The merged school would be sited in a new building on the current St Andrew's Primary School site. When the Scottish government calls in any proposal to close a school, it examines the process followed by the council and the information used to reach the decision. But it cannot simply overturn a decision because it disagrees with it. A letter informing the council of the government's decision said ministers were concerned by allegations the council's consultation document contained inaccurate information. It also said concerns raised by Education 

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



...

Input: summarize: Tawel Fan ward at Glan Clwyd Hospital, Denbighshire, was closed more than three years ago and a report found some patients were treated like animals. It has emerged that at least seven patients' families were told treatment may have contributed to their deaths. Betsi Cadwaladr health board said an investigation was under way. It acknowledged the quality of care provided could have been a contributory factor to the deaths of some patients. A review of mortality rates on the ward has never been published although it is understood it has been completed. Relatives of one patient told BBC Wales Today they were told medical care on the ward was inadequate. Correspondence seen by the programme included an apology from the health board to the family, who do not want to be identified. One letter said: "Experts found that there were problems in the health care which may have contributed to the death." It added that "the board is very much engaged in a thorough search for the tr

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



Input: summarize: Declan Paul Butcher from Roe Park Court committed the'single punch' assaults on Market Street in Limavady on 2 October 2014. CCTV footage showed his first victim being knocked unconscious. A second man suffered a double jaw fracture. Butcher appeared at Londonderry Crown Court on Thursday. He was sentenced to 14 months in prison. The court heard that the defendant had 51 previous convictions including eight assaults. He had also been assessed as presenting a high likelihood of reoffending. The judge told Butcher he had inflicted "gratuitous violence" on two" completely innocent" members of the public. The second man was punched and knocked out when he came across the first victim lying unconscious on Market Street. Both men later regained consciousness in hospital.
Output: A man has been jailed for a series of assaults on two people in Londonderry.
Label: A 25-year-old man from Limavady has been jailed for knocking out two men in separate attacks on the same night.
--

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



Input: summarize: "Hope made a poor decision that has resulted in a negative impact on US Soccer and her team-mates," head coach Jill Ellis said. Solo, 33, will now miss matches on 8 February against France and 13 February against England. This month, domestic violence charges against Solo were dropped. In 2012, Solo, tested positive for a banned substance a month before winning a second Olympic gold medal. She maintained she was not aware the pre-menstrual medication she had been prescribed contained a banned substance and she was cleared of any wrongdoing. The US are preparing for the Women's World Cup in Canada in June when they will aim to win their third title following victories in 1991 and 1999. The latest incident surrounding Solo concerns a training camp being held by the team in Carson, California. Solo's husband, former Seattle Seahawks NFL player Jerramy Stevens, was arrested on suspicion of driving under the influence in Los Angeles in the early hours of Monday morning. Me

In [ ]:
import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# Metric
metric = evaluate.load("rouge")

rogue = metric.compute(predictions=predictions, references=references, use_stemmer=True)
print(f"Rogue1: {rogue['rouge1']* 100:2f}%")
print(f"rouge2: {rogue['rouge2']* 100:2f}%")
print(f"rougeL: {rogue['rougeL']* 100:2f}%")
print(f"rougeLsum: {rogue['rougeLsum']* 100:2f}%")

Rogue1: 39.653424%
rouge2: 11.581853%
rougeL: 29.455628%
rougeLsum: 29.246060%


In [ ]:
model.save_pretrained("model")

Configuration saved in model/config.json
Configuration saved in model/generation_config.json
Model weights saved in model/pytorch_model.bin


In [ ]:
# #Training data
# import pandas as pd
# from datasets import Dataset, concatenate_datasets
# df_train = pd.read_csv('/content/df_train.csv')
# df_test = pd.read_csv('/content/df_test.csv')
# df_train = Dataset.from_pandas(df_train)
# df_test = Dataset.from_pandas(df_test)
# my_dict = {'train':  df_train,
#            'test': df_test
#            }

# # #dataset = concatenate_datasets([df_train, df_test])
# # dataset = Dataset.from_dict(my_dict)

In [ ]:
# from datasets import load_dataset
# dataset = load_dataset('csv', data_files={'train': 'df_train.csv',
#                                               'test': 'df_test.csv'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:

# # df_test = pd.DataFrame(dataset)
# # df_test = df_test.dropna()
# # print(df_test.head())

In [ ]:
# from datasets import load_dataset
# import pandas as pd
# dataset = load_dataset("xsum")
# cols = ['document','summary']
# df_test = pd.DataFrame(dataset['test'])
# df_test = df_test.dropna()
# df_test = df_test.reset_index()
# df_test = df_test[0:100]
# df_test = df_test[cols]
# df_train = pd.DataFrame(dataset['train'])
# df_train = df_train.dropna()
# df_train = df_train[0:1000]
# df_train = df_train[cols]
# df_train = df_train.reset_index()

# df_train.to_csv('df_train.csv', index= False)
# df_test.to_csv('df_test.csv', index= False)


# print(df_test.head())